# Leçon 1 : Deviner la suite

## Comment une IA "devine" le mot suivant ?

Quand tu écris un message sur ton téléphone, il te **propose** le mot suivant.
Comment fait-il ? Il a appris quels mots viennent souvent après d'autres.

On va faire pareil, mais avec des **lettres** au lieu de mots.

---

### Exercice 1 : À toi de deviner

Complète ces suites :
- A, B, C, **?**
- L, U, N, D, **?**
- Après la lettre Q, quelle lettre vient presque toujours ? **?**

Tu viens de faire ce que fait une IA : **prédire la suite** en utilisant des patterns que tu connais.

## Compter les lettres qui suivent

On va commencer simplement : prendre une liste de prénoms et compter
quelle lettre vient après quelle autre.

In [ ]:
# Nos prénoms d'entraînement
prenoms = [
    "emma", "lucas", "lea", "hugo", "chloe",
    "louis", "alice", "jules", "lina", "adam",
    "rose", "arthur", "manon", "paul", "jade",
    "nathan", "eva", "leo", "clara", "noah",
]

print(f"On a {len(prenoms)} prénoms pour apprendre.")
print("Exemples :", prenoms[:5])

In [ ]:
# Comptons : après chaque lettre, quelle lettre vient ensuite ?
from collections import Counter

# On utilise "." pour marquer le début et la fin d'un prénom
compteur = {}

for prenom in prenoms:
    # On ajoute un point au début et à la fin
    mot = "." + prenom + "."
    for i in range(len(mot) - 1):
        lettre_actuelle = mot[i]
        lettre_suivante = mot[i + 1]
        if lettre_actuelle not in compteur:
            compteur[lettre_actuelle] = Counter()
        compteur[lettre_actuelle][lettre_suivante] += 1

# Que vient-il après la lettre 'a' ?
print("Après la lettre 'a', on trouve :")
for lettre, nb in compteur['a'].most_common():
    print(f"  '{lettre}' -> {nb} fois")

## Transformer les comptes en probabilités

Au lieu de dire "la lettre 'r' vient 3 fois après 'a'",
on veut dire "il y a 25% de chances que 'r' vienne après 'a'".

C'est ce qu'on appelle une **probabilité**.

In [ ]:
# Transformer les comptes en probabilités
probas = {}

for lettre, suivantes in compteur.items():
    total = sum(suivantes.values())
    probas[lettre] = {}
    for suivante, nb in suivantes.items():
        probas[lettre][suivante] = nb / total

# Probabilités après 'a'
print("Probabilités après 'a' :")
for lettre, p in sorted(probas['a'].items(), key=lambda x: -x[1]):
    barre = '#' * int(p * 40)
    print(f"  '{lettre}' : {p:.0%} {barre}")

## Générer un prénom !

Maintenant on peut **inventer** un prénom :
1. On part du début (le point `.`)
2. On choisit la lettre suivante au hasard, en respectant les probabilités
3. On recommence jusqu'à tomber sur un point `.` (fin du prénom)

In [ ]:
import random

def generer_prenom(probas):
    """Génère un prénom lettre par lettre."""
    prenom = ""
    lettre = "."  # on commence au début

    while True:
        # Les choix possibles et leurs probabilités
        choix = list(probas[lettre].keys())
        poids = list(probas[lettre].values())

        # Choisir au hasard (mais les lettres fréquentes ont plus de chances)
        lettre = random.choices(choix, weights=poids, k=1)[0]

        if lettre == ".": # fin du prénom
            break
        prenom += lettre

    return prenom

# Générons 10 prénoms !
print("Prénoms inventés par notre modèle :")
print()
for i in range(10):
    print(f"  {generer_prenom(probas).capitalize()}")

## Ce qu'on a appris

- Un modèle de langage **prédit la suite** en se basant sur ce qu'il a vu avant
- Il utilise des **probabilités** : certaines lettres sont plus probables que d'autres
- Même un modèle très simple peut générer des mots qui "sonnent" bien

### Problème

Notre modèle ne regarde que **1 lettre en arrière**. Il ne sait pas que
"Chl" est un bon début de prénom. Dans la prochaine leçon, on va lui
apprendre à regarder plus loin en arrière et à **s'améliorer** quand il se trompe.

---
*Prochaine leçon : [02 - Apprendre de ses erreurs](02_apprendre_des_erreurs.ipynb)*

---

### Sources (ISO 42001)

- **Concept de bigrammes et modèles de langage** : [microgpt.py](https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95) — Andrej Karpathy
- **Approche pédagogique character-level** : [Vidéo "Let's build GPT"](https://www.youtube.com/watch?v=kCc8FmEb1nY) — Andrej Karpathy (2023)
- **Visualisation des réseaux de neurones** : [3Blue1Brown - Neural Networks](https://www.youtube.com/playlist?list=PLZHQObOWTQDNU6R1_67000Dx_ZCJB-3pi) — Grant Sanderson